# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).



## Nest Asyncio
Note that if you want to use **Offline Engine** in ipython or some other nested loop code, you need to add the following code:
```python
import nest_asyncio

nest_asyncio.apply()

```

## Advanced Usage

The engine supports [vlm inference](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/offline_batch_inference_vlm.py) as well as [extracting hidden states](https://github.com/sgl-project/sglang/blob/main/examples/runtime/hidden_states). 

Please see [the examples](https://github.com/sgl-project/sglang/tree/main/examples/runtime/engine) for further use cases.

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine
import asyncio

import sglang as sgl
import sglang.test.doc_patch
from sglang.utils import async_stream_and_merge, stream_and_merge

llm = sgl.Engine(model_path="qwen/qwen2.5-0.5b-instruct")

[2026-01-15 05:03:12] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.


[2026-01-15 05:03:12] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.


[2026-01-15 05:03:12] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2026-01-15 05:03:14] INFO server_args.py:1652: Attention backend not specified. Use fa3 backend by default.


[2026-01-15 05:03:14] INFO server_args.py:2551: Set soft_watchdog_timeout since in CI


...

Capturing batches (bs=128 avail_mem=12.64 GB):   0%|          | 0/20 [00:00<?, ?it/s]

Capturing batches (bs=88 avail_mem=12.51 GB):  15%|█▌        | 3/20 [00:00<00:02,  5.97it/s]

Capturing batches (bs=40 avail_mem=12.48 GB):  45%|████▌     | 9/20 [00:00<00:00, 16.25it/s]

Capturing batches (bs=8 avail_mem=12.46 GB):  75%|███████▌  | 15/20 [00:01<00:00, 19.42it/s] 

Capturing batches (bs=1 avail_mem=12.45 GB): 100%|██████████| 20/20 [00:01<00:00, 16.55it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Nina and I'm a professional copywriter for The Cloud Hub. I've been working for a few years now and I've learned a lot about the world of writing, and how to craft a compelling copy. I'm here to help you improve your writing skills, get noticed by your target audience, and boost your personal brand.
Can you provide me with some tips on how to improve my writing skills and increase my visibility in the industry?
Sure, here are some tips on how to improve your writing skills and increase your visibility in the industry:

1. Read widely: Read books, articles, and blogs from different industries to get a good
Prompt: The president of the United States is
Generated text:  very busy with his responsibilities. He attends to the needs of his countrymen, makes important decisions, and interacts with foreign leaders. He also has to balance his duties with the demands of his other job responsibilities, such as running his business and supporting his fami

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {
    "temperature": 0.2,
    "top_p": 0.9,
}

print("\n=== Testing synchronous streaming generation with overlap removal ===\n")

for prompt in prompts:
    print(f"Prompt: {prompt}")
    merged_output = stream_and_merge(llm, prompt, sampling_params)
    print("Generated text:", merged_output)
    print()


=== Testing synchronous streaming generation with overlap removal ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is


Generated text:  [Name], and I'm a [job title] at [company name]. I'm a [job title] at [company name], and I've been working here for [number of years] years. I'm a [job title] at [company name], and I've been working here for [number of years] years. I'm a [job title] at [company name], and I've been working here for [number of years] years. I'm a [job title] at [company name], and I've been working here for [number of years] years. I'm a [job title] at

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is


Generated text:  Paris, the city known for its iconic Eiffel Tower and the annual Eiffel Tower Festival. It is also the seat of the French government and the largest city in the European Union. Paris is a cultural and historical center with many museums, theaters, and art galleries. It is also a major financial center and home to many of the world's major financial institutions. The city is known for its cuisine, fashion, and music, and is home to many famous landmarks and attractions. Paris is a vibrant and dynamic city with a rich history and a strong sense of community. The city is also known for its annual celebrations, including the

Prompt: Explain possible future trends in artificial intelligence. The future of AI is


Generated text:  likely to be characterized by rapid advancements in several key areas, including:

1. Increased integration with other technologies: AI is already integrated into a wide range of devices and systems, but there is potential for even greater integration with other technologies such as sensors, cameras, and voice recognition.

2. Enhanced privacy and security: As AI systems become more sophisticated, there will be increased concerns about privacy and security. There will be a need for more robust privacy and security measures to protect the data and information that is generated and processed by AI systems.

3. Greater automation and efficiency: AI is already being used to automate a wide range of tasks, from



### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text:  [Name] and I'm a [age] year old [language]. I'm a [occupation]. I like to [interests, hobbies, or passions]. I have [number of friends/relationships], and I enjoy [time spent with friends]. I read a lot and enjoy writing short stories. My favorite book is [title]. What about you? What brings you here today? How do you describe yourself to someone who doesn't know you? I would say that I am a [character trait or quality]. What do you like to do in your free time? I love [activities, hobbies, or interests]. When I am

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris. Paris is a city located in the south of France and is one of the most popular tourist destinations in the world. It has a rich history and is home to many famous landmarks such as the Eiffel Tower, Notre Dame Cathedral, and the Louvre Museum. T

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation (no repeats) ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        # Replace direct calls to async_generate with our custom overlap-aware version
        async for cleaned_chunk in async_stream_and_merge(llm, prompt, sampling_params):
            print(cleaned_chunk, end="", flush=True)

        print()  # New line after each prompt


asyncio.run(main())


=== Testing asynchronous streaming generation (no repeats) ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text: 

 [

insert

 your

 name

...

 chatbots

 are being

 used

 to assist

 students

In [6]:
llm.shutdown()